In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))


TensorFlow version: 2.20.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Dataset ---
img_size = (224, 224)
batch_size = 32
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets",
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets",
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Kelas:", class_names)

# --- Optimasi pipeline (prefetch) ---
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- Augmentasi ---
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# --- Base Model Transfer Learning ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze dulu

# --- Model ---
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# --- Training ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# --- Evaluasi ---
loss, acc = model.evaluate(val_ds)
print(f"\n✅ Akurasi validasi: {acc:.2f}")

# --- Simpan model ---
model.save("bicara_mobilenetv2.h5")


Found 4000 files belonging to 4 classes.
Using 3200 files for training.
Found 4000 files belonging to 4 classes.
Using 800 files for validation.
Kelas: ['C', 'I', 'L', 'V']
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.4581 - loss: 1.2394 - val_accuracy: 0.8800 - val_loss: 0.6143
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.7816 - loss: 0.6310 - val_accuracy: 0.9850 - val_loss: 0.3169
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.9028 - loss: 0.3722 - val_accuracy: 0.9962 - val_loss: 0.1938
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9531 - loss: 0.2543 - val_accuracy: 0.9975 - val_loss: 0.1251
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9719 - loss: 0.1865 - val_accuracy: 0.9975 - val_loss: 0.0913
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.9819 - loss: 0.1460 - val_accuracy: 0.9975 - val_loss: 0.0696
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step


✅ Akurasi validasi: 1.00


In [18]:
# 1. Import library
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib
from tensorflow.keras.preprocessing import image

# 2. Path dataset dan model
dataset_dir = pathlib.Path('/home/antariksa/Desktop/Coding/bicara-model/datasets')
model_path = "bicara_mobilenetv2.h5"

# 3. Load model yang sudah dilatih
model = keras.models.load_model(model_path)
print("✅ Model berhasil dimuat!")

# 4. Ambil nama kelas dari folder dataset (sama dengan saat training)
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

class_names = train_ds.class_names
print("Kelas yang ditemukan:", class_names)

def predict_image(img_path):
    # Load gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # JANGAN dibagi 255 (sudah ada Rescaling di model)

    # Prediksi (pastikan dropout nonaktif)
    predictions = model(img_array, training=False).numpy()
    score = tf.nn.softmax(predictions[0])

    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    print(f"Gambar: {img_path}")
    print(f"Prediksi: {predicted_class} ({confidence:.2f}% confidence)")

    return predicted_class, confidence


# 6. Contoh penggunaan
img_path = "/home/antariksa/Desktop/Coding/bicara-model/datasets/V/Image_900.jpg"  # ganti sesuai gambar
predict_image(img_path)


✅ Model berhasil dimuat!
Found 4000 files belonging to 4 classes.
Using 3200 files for training.
Kelas yang ditemukan: ['C', 'I', 'L', 'V']
Gambar: /home/antariksa/Desktop/Coding/bicara-model/datasets/V/Image_900.jpg
Prediksi: V (43.88% confidence)


('V', 43.879154324531555)

In [19]:
import cv2
import numpy as np
import tensorflow as tf

# --- Load model ---
model = tf.keras.models.load_model("bicara_mobilenetv2.h5")

# Daftar kelas sesuai urutan dataset
class_names = ['C', 'I', 'L', 'V']
img_size = (224, 224)

# --- Buka kamera ---
cap = cv2.VideoCapture(0)  # 0 = webcam default

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocessing frame
    img = cv2.resize(frame, img_size)
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    # Prediksi
    predictions = model.predict(img)
    score = tf.nn.softmax(predictions[0])
    class_id = np.argmax(score)
    confidence = 100 * np.max(score)

    # Tampilkan hasil di frame
    label = f"{class_names[class_id]}: {confidence:.2f}%"
    cv2.putText(frame, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Bicara Model - Kamera", frame)

    # Tombol 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━